In [1]:
import glob
txt_files = glob.glob("reviews_corpus/*.txt")

In [2]:
reviews = [open(txt,"r").read() for txt in txt_files]

In [3]:
reviews

['"Deathstalker" of 1983 was an incredibly crappy movie, but in its own way incredibly entertaining and fun to watch, simply due to its value as an unintentional comedy. The first sequel, "Deathstalker II: Duel Of The Titans" of 1987 almost seems to be a spoof of its predecessor. As the first movie, "Deathstalker 2" is again extremely crappy and therefore quite funny, but it seems that director Jim Wynorski as well as the actors and everybody else connected with this film were actually willing to do a movie as crappy as possible on purpose. While the Deathstalker was basically a brutal and invincible Barbarian cheapo version of Conan played by muscleman Rick Hill in the first movie, the title character became a skinny guy with a tendency towards stupid jokes. The dialogue is even sillier and out of place than in the first movie ("a cigar for the lady", "is that your sword or are you just happy to see me"...), the acting is (intentionally?) terrible and hardly anything in the movie make

In [4]:
text = "".join(reviews)

In [5]:
chars_list = sorted(list(set(text)))
charToIndexDict = {character: chars_list.index(character) for character in chars_list}

In [6]:
charToIndexDict

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '/': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 ';': 18,
 '<': 19,
 '>': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71}

In [62]:
import keras
from keras import layers
maxLength=70
rnn = keras.models.Sequential()
rnn.add(layers.LSTM(1024, input_shape=(maxLength, len(chars_list)),return_sequences=True))
rnn.add(layers.LSTM(1024, input_shape=(maxLength, len(chars_list))))
rnn.add(layers.Dense(len(chars_list), activation='softmax'))

In [63]:
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, nesterov=True) 
rnn.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [64]:
import numpy as np
def textToVector(input_txt, maxLength):
    sentences = []
    next_characters = []
    for i in range(0, len(input_txt) - maxLength):
        sentences.append(input_txt[i : i + maxLength])
        next_characters.append(input_txt[i + maxLength])
    X = np.zeros((len(sentences), maxLength, len(chars_list)))
    y = np.zeros((len(sentences), len(chars_list)))
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, charToIndexDict[char]] = 1
            y[i, charToIndexDict[next_characters[i]]] = 1
    return [X, y]

In [65]:
X,y = textToVector(text, maxLength)

In [ ]:
rnn.fit(X, y, batch_size=256, epochs=1)

Epoch 1/1
29440/35180 [========================>.....] - ETA: 3:50 - loss: 3.3581

In [67]:
rnn.save_weights('weights.hdf5')